## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

week_ending = "Dec. 1st"
week_starting = "Nov. 24th"

print(week_starting, week_ending)

Nov. 24th Dec. 1st


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-25-2024,Condo/Co-op,610 NE 2nd St #205,Dania Beach,FL,33004.0,532000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,MARMLS,A11699941,N,Y,26.055795,-80.133206


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
43,PAST SALE,2024-11-29,Condo/Co-op,6020 NW 44th St #106,Lauderhill,FL,33319.0,169900.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/6020-NW-4...,Beaches MLS,F10433429,N,Y,26.178925,-80.232356
56,PAST SALE,2024-11-28,Condo/Co-op,10024 Winding Lake Rd #103,Sunrise,FL,33351.0,290000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/10024-Windin...,MARMLS,A11594952,N,Y,26.173375,-80.287561
31,PAST SALE,2024-11-27,Condo/Co-op,9410 Tangerine Pl #301,Davie,FL,33324.0,194000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Davie/9410-Tangerine...,MARMLS,A11661890,N,Y,26.101666,-80.274731
16,PAST SALE,2024-11-27,Condo/Co-op,1256 S Military Trl #923,Deerfield Beach,FL,33442.0,222000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1256...,Beaches MLS,RX-11028458,N,Y,26.300923,-80.130755
37,PAST SALE,2024-11-27,Condo/Co-op,1705 Belmont Ln #1705,North Lauderdale,FL,33068.0,240000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Lauderdale/170...,MARMLS,A11626308,N,Y,26.206033,-80.233284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,PAST SALE,2024-11-25,Condo/Co-op,1116 N Ocean Blvd #1803,Pompano Beach,FL,33062.0,2300000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11640073,N,Y,26.247536,-80.086777
49,PAST SALE,2024-11-25,Condo/Co-op,305 N Pompano Beach Blvd Ph 09,Pompano Beach,FL,33062.0,590000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/305-N-...,Beaches MLS,F10455580,N,Y,26.236771,-80.089682
50,PAST SALE,2024-11-25,Condo/Co-op,3901 S Ocean Dr Unit 14J,Hollywood,FL,33019.0,470000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3901-S-Oce...,MARMLS,A11635494,N,Y,25.988153,-80.118301
54,PAST SALE,2024-11-25,Condo/Co-op,5539 Courtyard Dr #5539,Margate,FL,33063.0,188000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/5539-Courtya...,Beaches MLS,F10441646,N,Y,26.266860,-80.198708


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

83


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$388,858


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$301


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$32,275,200


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
119
Input Previous Week Condo Average Sales Price:
288204
Input Previous Week Condo Average PSF:
261
Input Previous Week Condo Sales total (ex: 198_000_000)
34_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-27,Condo/Co-op,1116 N Ocean Blvd #803,Pompano Beach,FL,33062.0,4500000.0,4.0,4.5,Solemar,4096.0,NaN,2023.0,NaN,1099.0,5881.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11520726,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-27,Condo/Co-op,1116 N Ocean Blvd #803,Pompano Beach,FL,33062.0,4500000.0,4.0,4.5,Solemar,4096.0,NaN,2023.0,NaN,1099.0,5881.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11520726,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",1,orange
1,PAST SALE,2024-11-25,Condo/Co-op,1116 N Ocean Blvd #1803,Pompano Beach,FL,33062.0,2300000.0,2.0,2.5,Solemar,1885.0,NaN,2023.0,NaN,1220.0,2702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11640073,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Megan Probst,"Compass Florida, LLC",Crystal Moonilal,"Compass Florida, LLC",None,None,None,None,2,blue
2,PAST SALE,2024-11-27,Condo/Co-op,2100 S Ocean Ln #1206,Fort Lauderdale,FL,33316.0,1420000.0,3.0,4.0,POINT OF AMERICAS,2524.0,NaN,1969.0,NaN,563.0,2676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,MARMLS,A11493971,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue
3,PAST SALE,2024-11-26,Condo/Co-op,4100 Galt Ocean Dr #912,Fort Lauderdale,FL,33308.0,1350000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,466.0,1933.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10451324,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Kevin Wirth,Castelli Real Estate Services,None,None,Kevin Wirth,Castelli Real Estate Services,None,None,4,blue
4,PAST SALE,2024-11-25,Condo/Co-op,2051 SE 3rd St #505,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2003.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11005695,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Jennifer Gallagher,Serhant,None,None,Jane Albright,Coldwell Banker Realty,None,None,5,blue
5,PAST SALE,2024-11-27,Condo/Co-op,111 Briny Ave Ph 3,Pompano Beach,FL,33062.0,770000.0,2.0,2.5,Pompano Beach Club Condo,1235.0,NaN,1974.0,NaN,623.0,1194.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Beaches MLS,RX-10967986,N,Y,26.230531,-80.090146,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,6,blue
6,PAST SALE,2024-11-25,Condo/Co-op,610 NE 2nd St #301,Dania Beach,FL,33004.0,756600.0,2.0,2.0,Atlantica,1159.0,NaN,2024.0,NaN,653.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,MARMLS,A11699948,N,Y,26.055795,-80.133206,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,Craig Studnicky,ISG Dania Beach LLC,None,None,Juan Alvarez,Blokhaus Real Estate + Investments Inc,None,None,7,blue
7,PAST SALE,2024-11-26,Condo/Co-op,1800 S Ocean Dr #3409,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB THREE CONDO,1086.0,NaN,2006.0,NaN,645.0,1224.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/180...,MARMLS,A11677400,N,Y,25.985667,-80.118693,https://www.redfin.com/FL/Hallandale-Beach/180...,Brian Feivelson,LPT Realty,Steven Pesso,LPT Realty,None,None,None,None,8,blue
8,PAST SALE,2024-11-25,Condo/Co-op,305 N Pompano Beach Blvd Ph 09,Pompano Beach,FL,33062.0,590000.0,2.0,2.0,Jamaica House,1200.0,NaN,1969.0,NaN,492.0,655.0,Sold,NaN,NaN,https://www.redfin.com/FL/

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-27,Condo/Co-op,1116 N Ocean Blvd #803,Pompano Beach,FL,33062.0,4500000.0,4.0,4.5,Solemar,4096.0,NaN,2023.0,NaN,1099.0,5881.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11520726,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_12022024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'Solemar at 1116 N Ocean Blvd #803 in Pompano Beach'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-27,Condo/Co-op,1116 N Ocean Blvd #803,Pompano Beach,FL,33062.0,4500000.0,4.0,4.5,Solemar,4096.0,NaN,2023.0,NaN,1099.0,5881.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11520726,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",1,orange,Solemar at 1116 N Ocean Blvd #803 in Pompano B...
1,PAST SALE,2024-11-25,Condo/Co-op,1116 N Ocean Blvd #1803,Pompano Beach,FL,33062.0,2300000.0,2.0,2.5,Solemar,1885.0,NaN,2023.0,NaN,1220.0,2702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11640073,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Megan Probst,"Compass Florida, LLC",Crystal Moonilal,"Compass Florida, LLC",None,None,None,None,2,blue,Solemar at 1116 N Ocean Blvd #1803 in Pompano ...
2,PAST SALE,2024-11-27,Condo/Co-op,2100 S Ocean Ln #1206,Fort Lauderdale,FL,33316.0,1420000.0,3.0,4.0,POINT OF AMERICAS,2524.0,NaN,1969.0,NaN,563.0,2676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,MARMLS,A11493971,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue,POINT OF AMERICAS at 2100 S Ocean Ln #1206 in ...
3,PAST SALE,2024-11-26,Condo/Co-op,4100 Galt Ocean Dr #912,Fort Lauderdale,FL,33308.0,1350000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,466.0,1933.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10451324,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Kevin Wirth,Castelli Real Estate Services,None,None,Kevin Wirth,Castelli Real Estate Services,None,None,4,blue,The Galleon at 4100 Galt Ocean Dr #912 in Fort...
4,PAST SALE,2024-11-25,Condo/Co-op,2051 SE 3rd St #505,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2003.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11005695,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Jennifer Gallagher,Serhant,None,None,Jane Albright,Coldwell Banker Realty,None,None,5,blue,Ocean Plaza at 2051 SE 3rd St #505 in Deerfiel...
5,PAST SALE,2024-11-27,Condo/Co-op,111 Briny Ave Ph 3,Pompano Beach,FL,33062.0,770000.0,2.0,2.5,Pompano Beach Club Condo,1235.0,NaN,1974.0,NaN,623.0,1194.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Beaches MLS,RX-10967986,N,Y,26.230531,-80.090146,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,6,blue,Pompano Beach Club Condo at 111 Briny Ave Ph 3...
6,PAST SALE,2024-11-25,Condo/Co-op,610 NE 2nd St #301,Dania Beach,FL,33004.0,756600.0,2.0,2.0,Atlantica,1159.0,NaN,2024.0,NaN,653.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,MARMLS,A11699948,N,Y,26.055795,-80.133206,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,Craig Studnicky,ISG Dania Beach LLC,None,None,Juan Alvarez,Blokhaus Real Estate + Investments Inc,None,None,7,blue,Atlantica at 610 NE 2nd St #301 in Dania Beach
7,PAST SALE,2024-11-26,Condo/Co-op,1800 S Ocean Dr #3409,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB THREE CONDO,1086.0,NaN,2006.0,NaN,645.0,1224.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/180...,MARMLS,A

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Solemar closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $580,000M to $4,500,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Dania Beach, Pompano Beach, Deerfield Beach, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $32,275,200 million from Nov. 24th to Dec. 1st. The previous week, brokers closed 119 condo sales totaling $34,300,000.

Last week’s units sold for an average of $388,858, higher than the $288,204 average price from the previous week. The average price per square 

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-27,Condo/Co-op,1116 N Ocean Blvd #803,Pompano Beach,FL,33062.0,4500000.0,4.0,4.5,Solemar,4096.0,NaN,2023.0,NaN,1099.0,5881.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11520726,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",1,orange,Solemar at 1116 N Ocean Blvd #803 in Pompano B...
1,PAST SALE,2024-11-25,Condo/Co-op,1116 N Ocean Blvd #1803,Pompano Beach,FL,33062.0,2300000.0,2.0,2.5,Solemar,1885.0,NaN,2023.0,NaN,1220.0,2702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11640073,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Megan Probst,"Compass Florida, LLC",Crystal Moonilal,"Compass Florida, LLC",None,None,None,None,2,blue,Solemar at 1116 N Ocean Blvd #1803 in Pompano ...
2,PAST SALE,2024-11-27,Condo/Co-op,2100 S Ocean Ln #1206,Fort Lauderdale,FL,33316.0,1420000.0,3.0,4.0,POINT OF AMERICAS,2524.0,NaN,1969.0,NaN,563.0,2676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,MARMLS,A11493971,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue,POINT OF AMERICAS at 2100 S Ocean Ln #1206 in ...
3,PAST SALE,2024-11-26,Condo/Co-op,4100 Galt Ocean Dr #912,Fort Lauderdale,FL,33308.0,1350000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,466.0,1933.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10451324,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Kevin Wirth,Castelli Real Estate Services,None,None,Kevin Wirth,Castelli Real Estate Services,None,None,4,blue,The Galleon at 4100 Galt Ocean Dr #912 in Fort...
4,PAST SALE,2024-11-25,Condo/Co-op,2051 SE 3rd St #505,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2003.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11005695,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Jennifer Gallagher,Serhant,None,None,Jane Albright,Coldwell Banker Realty,None,None,5,blue,Ocean Plaza at 2051 SE 3rd St #505 in Deerfiel...
5,PAST SALE,2024-11-27,Condo/Co-op,111 Briny Ave Ph 3,Pompano Beach,FL,33062.0,770000.0,2.0,2.5,Pompano Beach Club Condo,1235.0,NaN,1974.0,NaN,623.0,1194.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Beaches MLS,RX-10967986,N,Y,26.230531,-80.090146,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,6,blue,Pompano Beach Club Condo at 111 Briny Ave Ph 3...
6,PAST SALE,2024-11-25,Condo/Co-op,610 NE 2nd St #301,Dania Beach,FL,33004.0,756600.0,2.0,2.0,Atlantica,1159.0,NaN,2024.0,NaN,653.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,MARMLS,A11699948,N,Y,26.055795,-80.133206,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,Craig Studnicky,ISG Dania Beach LLC,None,None,Juan Alvarez,Blokhaus Real Estate + Investments Inc,None,None,7,blue,Atlantica at 610 NE 2nd St #301 in Dania Beach
7,PAST SALE,2024-11-26,Condo/Co-op,1800 S Ocean Dr #3409,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB THREE CONDO,1086.0,NaN,2006.0,NaN,645.0,1224.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/180...,MARMLS,A

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Solemar closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $580,000M to $4,500,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Dania Beach, Pompano Beach, Deerfield Beach, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $32,275,200 million from Nov. 24th to Dec. 1st. The previous week, brokers closed 119 condo sales totaling $34,300,000.

Last week’s units sold for an average of $388,858, higher than the $288,204 average price from the previous week. The average price per square 

In [55]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-803/home/189608935


In [49]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-1803/home/175050546


In [56]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Pompano-Beach/111-N-Pompano-Beach-Blvd-33062/unit-13/home/41526544


In [57]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Pompano-Beach/305-N-Pompano-Beach-Blvd-33062/unit-09/home/41526332


In [58]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-1803/home/175050546


In [59]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Pompano-Beach/111-N-Pompano-Beach-Blvd-33062/unit-13/home/41526544


In [62]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-27,Condo/Co-op,1116 N Ocean Blvd #803,Pompano Beach,FL,33062.0,4500000.0,4.0,4.5,Solemar,4096.0,NaN,2023.0,NaN,1099.0,5881.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11520726,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",Michelle Nunez,Fortune International Realty,Megan Probst,"Compass Florida, LLC",1,orange,Solemar at 1116 N Ocean Blvd #803 in Pompano B...
1,PAST SALE,2024-11-25,Condo/Co-op,1116 N Ocean Blvd #1803,Pompano Beach,FL,33062.0,2300000.0,2.0,2.5,Solemar,1885.0,NaN,2023.0,NaN,1220.0,2702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11640073,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Megan Probst,"Compass Florida, LLC",Crystal Moonilal,"Compass Florida, LLC",None,None,None,None,2,blue,Solemar at 1116 N Ocean Blvd #1803 in Pompano ...
2,PAST SALE,2024-11-27,Condo/Co-op,2100 S Ocean Ln #1206,Fort Lauderdale,FL,33316.0,1420000.0,3.0,4.0,POINT OF AMERICAS,2524.0,NaN,1969.0,NaN,563.0,2676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,MARMLS,A11493971,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Brian Hero,One Sotheby's Int'l Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,3,blue,POINT OF AMERICAS at 2100 S Ocean Ln #1206 in ...
3,PAST SALE,2024-11-26,Condo/Co-op,4100 Galt Ocean Dr #912,Fort Lauderdale,FL,33308.0,1350000.0,3.0,2.5,The Galleon,2900.0,NaN,1967.0,NaN,466.0,1933.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Beaches MLS,F10451324,N,Y,26.178986,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Kevin Wirth,Castelli Real Estate Services,None,None,Kevin Wirth,Castelli Real Estate Services,None,None,4,blue,The Galleon at 4100 Galt Ocean Dr #912 in Fort...
4,PAST SALE,2024-11-25,Condo/Co-op,2051 SE 3rd St #505,Deerfield Beach,FL,33441.0,1125000.0,3.0,2.5,Ocean Plaza,2056.0,NaN,2003.0,NaN,547.0,1457.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2051...,Beaches MLS,RX-11005695,N,Y,26.312078,-80.076524,https://www.redfin.com/FL/Deerfield-Beach/2051...,Jennifer Gallagher,Serhant,None,None,Jane Albright,Coldwell Banker Realty,None,None,5,blue,Ocean Plaza at 2051 SE 3rd St #505 in Deerfiel...
5,PAST SALE,2024-11-27,Condo/Co-op,111 Briny Ave Ph 3,Pompano Beach,FL,33062.0,770000.0,2.0,2.5,Pompano Beach Club Condo,1235.0,NaN,1974.0,NaN,623.0,1194.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Beaches MLS,RX-10967986,N,Y,26.230531,-80.090146,https://www.redfin.com/FL/Pompano-Beach/111-Br...,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,Melissa Jordan,Keller Williams Realty Boca Raton,None,None,6,blue,Pompano Beach Club Condo at 111 Briny Ave Ph 3...
6,PAST SALE,2024-11-25,Condo/Co-op,610 NE 2nd St #301,Dania Beach,FL,33004.0,756600.0,2.0,2.0,Atlantica,1159.0,NaN,2024.0,NaN,653.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,MARMLS,A11699948,N,Y,26.055795,-80.133206,https://www.redfin.com/FL/Dania-Beach/610-NE-2...,Craig Studnicky,ISG Dania Beach LLC,None,None,Juan Alvarez,Blokhaus Real Estate + Investments Inc,None,None,7,blue,Atlantica at 610 NE 2nd St #301 in Dania Beach
7,PAST SALE,2024-11-26,Condo/Co-op,1800 S Ocean Dr #3409,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB THREE CONDO,1086.0,NaN,2006.0,NaN,645.0,1224.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/180...,MARMLS,A

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 2, 2) ## List (Earlier) date
date2 = datetime(2024, 11, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

299


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [72]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-27,1116 N Ocean Blvd #803 in Pompano Beach,"$4,500,000",4,4.5,"4,096",2023,"$1,099",MARMLS,A11520726,Michelle Nunez Fortune International Realty Me...,Michelle Nunez Fortune International Realty Me...
1,2024-11-25,1116 N Ocean Blvd #1803 in Pompano Beach,"$2,300,000",2,2.5,"1,885",2023,"$1,220",MARMLS,A11640073,"Megan Probst Compass Florida, LLC Crystal Moon...",
2,2024-11-27,2100 S Ocean Ln #1206 in Fort Lauderdale,"$1,420,000",3,4.0,"2,524",1969,$563,MARMLS,A11493971,Brian Hero One Sotheby's Int'l Realty,Brian Hero One Sotheby's Int'l Realty
3,2024-11-26,4100 Galt Ocean Dr #912 in Fort Lauderdale,"$1,350,000",3,2.5,"2,900",1967,$466,Beaches MLS,F10451324,Kevin Wirth Castelli Real Estate Services,Kevin Wirth Castelli Real Estate Services
4,2024-11-25,2051 SE 3rd St #505 in Deerfield Beach,"$1,125,000",3,2.5,"2,056",2003,$547,Beaches MLS,RX-11005695,Jennifer Gallagher Serhant,Jane Albright Coldwell Banker Realty
5,2024-11-27,111 Briny Ave Ph 3 in Pompano Beach,"$770,000",2,2.5,"1,235",1974,$623,Beaches MLS,RX-10967986,Melissa Jordan Keller Williams Realty Boca Rat...,Melissa Jordan Keller Williams Realty Boca Rat...
6,2024-11-25,610 NE 2nd St #301 in Dania Beach,"$756,600",2,2.0,"1,159",2024,$653,MARMLS,A11699948,Craig Studnicky ISG Dania Beach LLC,Juan Alvarez Blokhaus Real Estate + Investment...
7,2024-11-26,1800 S Ocean Dr #3409 in Hallandale Beach,"$700,000",2,1.0,"1,086",2006,$645,MARMLS,A11677400,Brian Feivelson LPT Realty Steven Pesso LPT Re...,
8,2024-11-25,305 N Pompano Beach Blvd Ph 09 in Pompano Beach,"$590,000",2,2.0,"1,200",1969,$492,Beaches MLS,F10455580,Deborah London LoKation,Juan Gonzalez Galt Ocean Realty Inc
9,2024-11-26,111 N Pompano Beach Blvd Ph 13 in Pompano Beach,"$580,000",2,2.0,"1,450",1970,$400,Beaches MLS,F10465153,Paige Coburn Keller Williams Realty Profess,Onepath Realty LLC
